In [17]:
import torch 
import numpy as np
from botorch.models import SingleTaskGP
from botorch  import fit_gpytorch_model

from botorch.fit import fit_gpytorch_mll
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.acquisition import UpperConfidenceBound
from botorch.acquisition.monte_carlo import qExpectedImprovement

from botorch.optim import optimize_acqf

from sklearn.gaussian_process import GaussianProcessRegressor
from scipy.stats import norm
from scipy.optimize import minimize
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.gaussian_process.kernels import RBF

In [18]:
f6_inputs = np.load("initial_inputs.npy")
f6_outputs = np.load("initial_outputs.npy")
f6_inputs_2 = np.load("initial_inputs2.npy")
f6_outputs_2 = np.load("initial_outputs2.npy")

In [19]:
f6_inputs = np.concatenate((f6_inputs, f6_inputs_2)) 
f6_inputs = np.concatenate((f6_inputs, np.array([[0.345535, 0.754647, 0.491966, 0.450836, 0.604032]]))) 
f6_inputs = np.concatenate((f6_inputs, np.array([[0.853367, 0.617629, 0.937563, 0.301578, 0.415393]]))) 
f6_inputs = np.concatenate((f6_inputs, np.array([[0.688186, 0.184692, 0.772551, 0.691116, 0.066401]]))) 
f6_inputs = np.concatenate((f6_inputs, np.array([[0.622911, 0.51695 , 0.236728, 0.47113 , 0.706154]])))
f6_inputs = np.concatenate((f6_inputs, np.array([[0.668186, 0.204692, 0.792551, 0.711116, 0.086401]])))
f6_inputs = np.concatenate((f6_inputs, np.array([[0.648186, 0.224692, 0.812551, 0.691116, 0.066401]])))
f6_inputs = np.concatenate((f6_inputs, np.array([[0.648186, 0.184692, 0.812551, 0.731116, 0.106401]])))
f6_inputs = np.concatenate((f6_inputs, np.array([[0.628186, 0.164692, 0.792551, 0.751116, 0.126401]])))
f6_inputs = np.concatenate((f6_inputs, np.array([[0.668186, 0.204692, 0.832551, 0.751116, 0.126401]])))
f6_inputs = np.concatenate((f6_inputs, np.array([[0.381006, 0.349024, 0.584465, 0.808867, 0.155764]])))
f6_inputs = np.concatenate((f6_inputs, np.array([[0.427005, 0.337664, 0.600496, 0.733667, 0.176168]])))
f6_inputs = np.concatenate((f6_inputs, np.array([[0.363984, 0.316227, 0.577013, 0.689438, 0.077835]])))
f6_inputs = np.concatenate((f6_inputs, np.array([[0.459871, 0.365921, 0.606441, 0.783317, 0.239875]])))
f6_inputs = np.concatenate((f6_inputs, np.array([[0.447005, 0.317664, 0.620496, 0.713667, 0.196168]])))
f6_inputs = np.concatenate((f6_inputs, np.array([[0.435998, 0.406845, 0.668405, 0.811834, 0.130345]])))
f6_inputs = np.concatenate((f6_inputs, np.array([[0.45466 , 0.346872, 0.657419, 0.858968, 0.11944]])))
f6_inputs = np.concatenate((f6_inputs, np.array([[0.458766, 0.303592, 0.663674, 0.996131, 0.076816]])))
f6_inputs = np.concatenate((f6_inputs, np.array([[0.384916, 0.311188, 0.757855, 0.803686, 0.13494 ]])))

f6_outputs = np.concatenate((f6_outputs,f6_outputs_2)) 
f6_outputs = np.append(f6_outputs,-1.5810999457962531)
f6_outputs = np.append(f6_outputs,-1.8787437904480868)
f6_outputs = np.append(f6_outputs,-0.6247687161411921)
f6_outputs = np.append(f6_outputs,-1.718806647852857)
f6_outputs = np.append(f6_outputs,-0.5671773346921368)
f6_outputs = np.append(f6_outputs,-0.5874063902941494)
f6_outputs = np.append(f6_outputs,-0.5144914715079646)
f6_outputs = np.append(f6_outputs,-0.5415990286820394)
f6_outputs = np.append(f6_outputs,-0.5996653040294271)
f6_outputs = np.append(f6_outputs,-0.22072442707678375)
f6_outputs = np.append(f6_outputs,-0.18264958158407013)
f6_outputs = np.append(f6_outputs,-0.3440970432570667)
f6_outputs = np.append(f6_outputs,-0.23533067806486813)
f6_outputs = np.append(f6_outputs,-0.2639220819499121)
f6_outputs = np.append(f6_outputs,-0.17148052911247116)
f6_outputs = np.append(f6_outputs,-0.06247688853060665)
f6_outputs = np.append(f6_outputs,-0.4406374095498452)
f6_outputs = np.append(f6_outputs,-0.2633675699383991)


In [20]:
X_max= f6_inputs[np.argmax((f6_outputs))]
print(X_max)

[0.45466  0.346872 0.657419 0.858968 0.11944 ]


In [21]:
Y_max= f6_outputs[np.argmax((f6_outputs))]
print(Y_max)

-0.06247688853060665


In [24]:
variable_value = np.linspace(1, 10,10)
n_dims = 5 
beta = 0.001
np.random.seed(42)
train_X = torch.tensor(f6_inputs)
train_Y = torch.tensor(f6_outputs).reshape(len(f6_outputs),-1)
best_y = np.max(train_Y.numpy())
best_x = train_X[np.argmax(train_Y.numpy())]
noise_level =0.1
candidate_list = []
best_x_list = []

for i in range(len(variable_value)):
    #beta = variable_value[i]
   
    def function_6(x1,x2,x3,x4,x5, noise_level=noise_level):
        return np.sin(5 * x1+4*x2) * (1 - np.tanh(x1 ** 2) - np.tanh(x2)) + \
            np.sin(1 * x3+4*x4) * (1 - np.tanh(x3 ** 2) - np.tanh(x4)) + \
            np.sin(1 * x5+4*x2) * (1 - np.tanh(x5 ** 2) - np.tanh(x5)) + \
              + \
                   + np.random.randn() * noise_level
    
    
    gp = SingleTaskGP(train_X, train_Y)
    mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
    fit_gpytorch_mll(mll)

    #UCB = UpperConfidenceBound(gp, beta=beta)
    EI = qExpectedImprovement(model=gp , best_f=best_y)
    bounds = torch.stack([torch.zeros(n_dims), torch.ones(n_dims)])
    candidate, _  = optimize_acqf(
        EI, bounds=bounds, q=1, num_restarts=300, raw_samples=300,
    )


    x1,x2,x3,x4,x5 = candidate.numpy().ravel().tolist()
    # Get the next value for the function from the script

    y_next = function_6(x1,x2,x3,x4,x5)

    if y_next > best_y:
        best_y = y_next
        best_x = x1,x2,x3,x4,x5


    y_next = torch.tensor(y_next).unsqueeze(-1)

    train_X = torch.cat([train_X,candidate])
    train_Y = torch.cat([train_Y,y_next.unsqueeze(-1)])
    candidate_list.append(candidate) 
    best_x_list.append(best_x)
    #print(f"Next point: {candidate} \n Next Y {y_next}  Best_y, {best_y}")
    #print(f"beta :{beta} noise_level :{noise_level}")
    #print(f"Next point: {candidate} \n Next Y {y_next}")
    #print(f"best_x:{best_x}Best_y,{best_y}")
    

In [25]:
best_x_list

[(0.47977063059806824,
  0.4024425745010376,
  0.6558748483657837,
  0.8008358478546143,
  0.013618226163089275),
 (0.47977063059806824,
  0.4024425745010376,
  0.6558748483657837,
  0.8008358478546143,
  0.013618226163089275),
 (0.12503285706043243,
  0.035090524703264236,
  0.8703887462615967,
  0.4436342418193817,
  0.3652079403400421),
 (0.12503285706043243,
  0.035090524703264236,
  0.8703887462615967,
  0.4436342418193817,
  0.3652079403400421),
 (0.12503285706043243,
  0.035090524703264236,
  0.8703887462615967,
  0.4436342418193817,
  0.3652079403400421),
 (0.12503285706043243,
  0.035090524703264236,
  0.8703887462615967,
  0.4436342418193817,
  0.3652079403400421),
 (0.1694215089082718,
  0.0,
  0.9374870657920837,
  0.3059847056865692,
  0.3378622829914093),
 (0.1694215089082718,
  0.0,
  0.9374870657920837,
  0.3059847056865692,
  0.3378622829914093),
 (0.1694215089082718,
  0.0,
  0.9374870657920837,
  0.3059847056865692,
  0.3378622829914093),
 (0.1694215089082718,
  0.0,